In [ ]:
# pip install dash_bootstrap_components

In [1]:
import numpy as np
import pandas as pd
import requests, bs4
import os
import base64
import io
from datetime import datetime
from furl import furl
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [2]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
import dash_table
import plotly.express as px
import plotly.graph_objects as go
from dash.dependencies import Input, Output, State

<ipython-input-2-626c6b0820dd>:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-2-626c6b0820dd>:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-2-626c6b0820dd>:5: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table


In [3]:
def get_api(url):
    request = requests.get(url=url).text.encode('utf-8')
    xmlobj = bs4.BeautifulSoup(request, 'html.parser')
    rows = xmlobj.findAll('item')
    for i, row in enumerate(rows):
        climate.loc[len(climate)] = [
          row.tm.string.split(" ")[0],
          row.tm.string.split(" ")[1],
          row.ta.string,
          row.ws.string,
          row.wd.string,
          row.hm.string,
          row.pv.string,
          row.td.string,
          row.pa.string,
          row.ps.string,
          row.vs.string,
          row.ts.string ]
    return rows

In [4]:
def parse_contents(contents, filename, date):
    content_type, content_string = contents.split(',')

    decoded = base64.b64decode(content_string)
    try:
        if 'csv' in filename:
            # Assume that the user uploaded a CSV file
            df = pd.read_csv(
                io.StringIO(decoded.decode('utf-8')))
            df = df.head(5)
        elif 'xls' in filename:
            # Assume that the user uploaded an excel file
            df = pd.read_excel(io.BytesIO(decoded))
            df = df.head(10)
        with open(filename,"wb") as fp:
            fp.write(decoded)
    except Exception as e:
        print(e)
        return html.Div([
            'There was an error processing this file.'
        ])

    return html.Div([
        html.H5(filename),
        html.H6(datetime.fromtimestamp(date)),

        dash_table.DataTable(
            data=df.to_dict('records'),
            columns=[{'name': i, 'id': i} for i in df.columns],
        ),

        html.Hr()
    ])


In [5]:
def train_model_building(x,y):
    model = RandomForestRegressor()
    model.fit(x,y)
    return model.predict(x)

def train_model_factory(x,y):
    model = RandomForestRegressor()
    model.fit(x,y)
    return model.predict(x)


In [6]:
external_stylesheets = ["http://codepen.io/chriddyp/pen/bWLwgP.css"]
app = dash.Dash(__name__,external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.H5("계측량 예측 시스템"),
    html.Hr(),
    html.H6("계량정보데이터를 업로드하세요"),
    dcc.Upload(
        id='upload-data',
        children=html.Div([
            '파일을 드래그 하거나 ',
            html.A('클릭하여 선택하세요')
        ]),
        style={
            'width': '100%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '10px'
        },
        # Allow multiple files to be uploaded
        multiple=True
    ),
    html.Div(id='output-data-upload'),
    dcc.RadioItems(id="radio-item",
                  options = [
                      {"label":"건물","value":0},
                      {"label":"공장","value":1}]),
    html.Button('분석시작',id='button-id',n_clicks=0),
    html.Div(id='output-result')
])

In [9]:
@app.callback(Output('output-data-upload', 'children'),
              Input('upload-data', 'contents'),
              State('upload-data', 'filename'),
              State('upload-data', 'last_modified'))
def update_output(list_of_contents, list_of_names, list_of_dates):
    if list_of_contents is not None:
        children = [
            parse_contents(c, n, d) for c, n, d in
            zip(list_of_contents, list_of_names, list_of_dates)]
        return children
    
@app.callback(Output('output-result','children'),
              Input('button-id','n_clicks'),
              State('upload-data','filename'),
              State('radio-item','value'),
)

SyntaxError: unexpected EOF while parsing (<ipython-input-9-4981ea15796b>, line 16)

In [10]:
def update_result(n,filename,radio):
    filename = str(filename[0])
    if n >= 0 and radio == 0:
        df = pd.read_excel(filename,sheet_name='계량정보데이터')
        df=df.drop(['일자'],axis=1)
        df=df.drop(['Unnamed: 26'],axis=1)
        df = df.dropna()
        df=df.reset_index(drop=True,inplace=False)
        df.columns=['날짜','00:00','01:00','02:00','03:00','04:00','05:00','06:00','07:00','08:00','09:00','10:00','11:00','12:00','13:00','14:00','15:00','16:00','17:00','18:00','19:00','20:00','21:00','22:00','23:00']
        raw_all=pd.melt(df,id_vars=['날짜'])
        raw_all.columns=['DateTime','Hour','kwh']


        if raw_all.index.dtype == 'int64':
            raw_all.set_index('DateTime', inplace=True)
        raw_all=raw_all.reset_index()
        dt = raw_all.DateTime.str.slice(0,10)
        raw_all.drop('DateTime',axis=1,inplace=True)
        raw_all['dt'] = dt


        raw_all = raw_all[['dt','Hour','kwh']]
        raw_all = raw_all.sort_values(by=['dt','Hour']).reset_index(drop=True)

        climate = pd.DataFrame(columns=("dt","Hour","ta","ws","wd","hm","pv","td","pa","ps","vs","ts"))

        def get_api(url):
            request = requests.get(url=url).text.encode('utf-8')
            xmlobj = bs4.BeautifulSoup(request, 'html.parser')
            rows = xmlobj.findAll('item')
            for i, row in enumerate(rows):
                climate.loc[len(climate)] = [
                  row.tm.string.split(" ")[0],
                  row.tm.string.split(" ")[1],
                  row.ta.string,
                  row.ws.string,
                  row.wd.string,
                  row.hm.string,
                  row.pv.string,
                  row.td.string,
                  row.pa.string,
                  row.ps.string,
                  row.vs.string,
                  row.ts.string ]
            return rows